# Welcome!

Welcome to this special workshop on using Python for Geolocation of forensic artifacts! In this session we will explore a couple of online APIs that can be used to determine the physical location of a device or connection.

One easy way to determine a location is based upon the IP Address being used. IP Addresses are often found inside of logs on victim machines and forensics images. On victim machines these IP Addresses can be less than reliable because attackers will use various VPN services to hide their actual IP Address. We will discuss how to get around that problem later. For now, let's assume we are analyzing a forensics image  of the attacker machine or we have captured an early connection before the attacker began obfuscating their location. To determine their location, we can use an online website such as this one.

https://iplocation.com/

The main page of this website is not an API (Application Programming Interface).  It is an interface that was intended for a human being to interact with.  However, we can still determine the inputs and outputs of the program and have our script perform the same steps that a human would take.  This process is known as "Web Scraping". 
By viewing the source code on this website, we can determine how to interact with the website and use it automatically.

Let's walk through that process together.

Once we understand the website form, we can use Python to automatically retrieve information from the website. Websites have different terms of use so make sure you are limiting your interactions with the website to those terms.


In [ ]:
import requests
browser = requests.session()
response = browser.post("https://iplocation.com/", {"ip":"8.8.8.8"})
response.json()



Web scraping is not the most efficient way for our script to interact with hosts.   As a matter of fact, the response we just got from the https://iplocation.net website is much cleaner than we would usually expect to get back when web scraping. Usually, you have to clean up the response and filter out all of the data intended for human consumption. Ideally, we would interact with computers that designed to make it easy for programs rather than humans. Instead of scraping the website we could go through an API (Application Programming Interface). There are many different websites that offer APIs for IP location. A quick internet search will reveal many more than those listed here.

https://ipgeolocation.io/ 

http://iplocation.net 

https://ipstack.com/ 

https://ipinfodb.com/api 

https://ip-api.com/ 

https://www.ip2location.com/ 

Most require you to register for a free API key, but some do not. For example, the following website will permit you to make 15000 queries per hour without any registration. Copy and Paste the following link to an API into a new tab on this browser.  You will notice that the response is not very human friendly, but it is exactly what our programs would like to see.

https://freegeoip.app/json

Our script can query this API and get back results in a few simple lines of code.



In [ ]:
ip_address = "8.8.8.8"
result = browser.get(f"https://freegeoip.app/json/{ip_address}").json()
result


The variable 'result' contains a json object. Because it is in a computer friendly response we can access individual values in the response by using the .get() method.


In [ ]:
print(f"The IP Address is {result.get('ip')}")
print(f"The country is {result.get('country_name')}")


With a little bit of Python skill, you can extract large numbers of IP addresses from logs and other forensics artifacts then use these services to lookup their location. How to extract those IP addresses is beyond the scope of this workshop but is covered extensively in SEC573 Automating Information Security with Python. Once you have those IP addresses a simple loop can resolve those locations for you. Here is a bit of code to build a list of IP Addresses from a DNS Servers query log.


In [ ]:
import pathlib
import re
log_data = pathlib.Path("./dnslog.txt").read_text()
list_of_addresses = re.findall(r"client (\S+?)#", log_data)[:100]
list_of_addresses


Now, the variable list_of_addresses contains the first 100 IP Addresses inside of this DNS servers log file. We limited it to only 100 IP Addresses with Python's slicing syntax [:100]. The reasons we limited it here are two-fold. First, 100 addresses are more than enough to get a sense of how it works. Second, we don't want our workshop to consume a lot of their free service unnecessarily. Now that we have a list of IP Addresses, we can look them all up with a small loop.


In [ ]:
for each_ip in list_of_addresses:
    url = f"https://freegeoip.app/json/{each_ip}"
    result = browser.get(url).json()
    print(f"The country for IP Address {result.get('ip')} is {result.get('country_name')}")


Notice the speed is less than ideal. We can see each request as it goes to the server and we wait for the response. However, the data is useful. Let's add three lines of code to help categorize this data. These three lines use another trick we cover in SEC573 and create a "default dictionary" to group together our IP Addresses by the country of origin.

When you run the next cell, it will take the same amount of time or less than it did above. Printing things is a slow process for computers so by only printing at the end we can speed up the process. With 100 IP Addresses you may not notice, but with 1000s of addresses you will be glad you knew this. You will notice that response contains a group of IP Addresses that do not have any associated country. These are IP addresses for which the http://freegeoip.app website has no known location information.


In [ ]:
from collections import defaultdict
by_country = defaultdict(lambda :[])
print("Processing.  Please wait...")
for each_ip in list_of_addresses:
    url = f"https://freegeoip.app/json/{each_ip}"
    result = browser.get(url).json()
    by_country[result.get('country_name')].append(result.get('ip'))
by_country    


You can see that with very little code you can look up the location of IP addresses with online resources. This is useful for small sets of data but if you have a large amount of data there are faster ways.   We will address this problem in our next section.

